# 발표 소단원 리스트

12-2,12-6,12-10,12-14

# 문항별 자료 
- (12-2) thread가 작동 중인지 알아보기

- (12-6) Thread specific(local)한 데이터 저장 방법

- (12-10) Actor task 정의하기 : concurrency

- (12-14) Daemon Process 만들기

### (12-2) Determinig if a thread has started
- (sol) threading 라이브러리에 Event 객체 사용

In [16]:
## thread control
from threading import Thread,Event
import time

def countdown(n,evt):
    for i in range(n):
        print(i)
        time.sleep(1)
    
    evt.set() 
    
    while n > 0:
        print("T-minus",n)
        n-=1
        time.sleep(1)
        
started_evt = Event()
t = Thread(target=countdown,args=(10,started_evt))
t.start()

print("main thread works")
time.sleep(3)
print("main thread after 3 sec")

started_evt.wait() # let thread stops working until meet set
print("Countdown will start")

t.join()
print("Thread finishes working")

0main thread works

1
2
main thread after 3 sec
3
4
5
6
7
8
9
T-minusCountdown will start 
10
T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1
Thread finishes working


In [ ]:
## event object suits for one-time event
## for multiple times -> using Condition object

1

In [3]:
import threading
import time

class PeriodicTimer:
    def __init__(self,interval):
        self._interval = interval
        self._flag = 0
        self._cv = threading.Condition()
        
    def start(self):
        t = threading.Thread(target = self.run)
        t.daemon = True
        t.start()
        
    def run(self):
        ## 계속 도는 중 -> interval 마다 thread wake up 시킴
        '''
        run the timer and notify waiting threads after each interval
        '''
        while True:
            time.sleep(self._interval)
            with self._cv:
                self._flag ^= 1 # 0^1 = 1
                self._cv.notify_all()
                
    def wait_for_tick(self):
        '''
        wait for the next tick of the timer
        '''
        with self._cv:
            last_flag = self._flag
            while last_flag == self._flag:
                self._cv.wait() ## notify_all 되기 전까지 wait하는 상태!!
                
ptimer = PeriodicTimer(5)
ptimer.start() # main thread

## Two threads that synchronize on the timer

def countdown(nticks):
    while nticks > 0:
        ptimer.wait_for_tick()
        print('T-minus',nticks)
        nticks -= 1
        
def countup(last):
    n = 0
    while n < last:
        ptimer.wait_for_tick()
        print('Counting',n)
        n += 1
        
threading.Thread(target = countdown,args=(10,)).start()
threading.Thread(target = countup,args=(5,)).start()

T-minusCounting 0
 10
CountingT-minus 9
 1
T-minusCounting 2
 8
CountingT-minus 7
 3
T-minusCounting 4 6

T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


In [27]:
import threading
def worker(n,sema):
    # wait to be signaled
    sema.acquire()
    print('Get item',n)
    
sema = threading.Semaphore(0)
nworkers = 10
for n in range(nworkers):
    t = threading.Thread(target = worker,args=(n,sema))
    t.start()

In [28]:
sema.release() # sema를 release해야 작동을 함!!

Get item 0


### (12-6) Storing Thread-Specific State
- (sol) threading.local() 사용

In [1]:
from socket import socket,AF_INET,SOCK_STREAM
import threading

class LazyConnection:
    def __init__(self,address,family = AF_INET, type_ = SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type_
        self.local = threading.local()
        
    def __enter__(self):
        if hasattr(self.local,'sock'):
            raise RuntimeError('Already Connected')
        self.local.sock = socket(self.family,self.type)
        self.local.sock.connect(self.address)
        return self.local.sock
    
    def __exit__(self,exc_ty,exc_val,tb):
        self.local.sock.close()
        del self.local.sock

In [3]:
from functools import partial
def test(conn):
    with conn as s:
        s.send(b'GET /index.html HTTP/1.0\r\n')
        s.send(b'Host: www.python.org\r\n')
        s.send(b'\r\n')
        resp = b''.join(iter(partial(s.recv,8192),b''))
        
    print('Got {} bytes \n'.format(len(resp)))
    
conn = LazyConnection(('www.python.org',80))
t1 = threading.Thread(target=test,args=(conn,))
t2 = threading.Thread(target=test,args=(conn,))
t1.start()
t2.start()
t1.join()
t2.join()

Got 392 bytes 

Got 392 bytes 



### (12-10) Defining an Actor Task
- (sol) "actor model" 

In [3]:
## thread control
from threading import Thread,Event
import time

In [2]:
class Actor_prev:
    def run(self):
        print("hello")
    

class Actor:
    def __init__(self):
        pass
        
    def _bootstrap(self,_other):
        try:
            #print("hello") ## 
            #self.run(_other)
            _other.run()
        except ActorExit:
            print("error occurs")
            pass
        finally:
            self._terminated.set()
            
    
    def start(self,_other):
        self._terminated = Event()
        t = Thread(target = self._bootstrap,args=(_other,))
        #print(_other)
        t.daemon = True
        t.start()
        
    def join(self):
        self._terminated.wait()
        
    def run(self,_other):
        for i in range(5):
            print(i)


In [4]:
a0 = Actor_prev()
a0.run()

hello


In [5]:
a0.run()

hello


In [7]:
a1 = Actor()
a1.start(a0)
a1.join()

hello


### (12.14) Launching a Daemon Process on Unix
- daemon 작업 설계하기
    - START와 STOP이 편해야